# Import libraries

In [ ]:
import pathlib
import pandas as pd
import plotly.express as px
import plotly.io as pio
# pio.templates.default = "plotly_white"

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.precision = 2

results_root = pathlib.Path("../results/")

all_stats_df = pd.DataFrame()

# Load the csv file with results into a pandas dataframe

In [ ]:
# sweep = "broad_sweep"
# boxplots_to_save = ["architecture"]
# interaction_plots_to_save = [
#     ("architecture", "gnn_layers"),
#     ("architecture", "mlp_layers"),
#     ("architecture", "hidden_channels"),
#     ("architecture", "pool"),
# ]

# sweep = "jk_pooling_sweep"
# boxplots_to_save = ["pool"]
# interaction_plots_to_save = [
#     ("pool", "architecture")
# ]

sweep = "regularizing_sweep"
boxplots_to_save = []
interaction_plots_to_save = [
    ("norm", "transform")
]

# sweep = "relaxed"

df = pd.read_csv(results_root / "runs_data" / f"{sweep}.csv")
df.head()

df = df.fillna(value="none")
df = df.rename(columns={"good_within.99": "≤ 1% error", "good_within.95": "≤ 5% error"})

# Draw boxplots to compare hyperparameter settings

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

hyperparameters = ["architecture", "gnn_layers", "mlp_layers", "hidden_channels", "jk", "pool", "activation", "norm", "transform", "dropout", "selected_features"]
metrics_to_plot = ["≤ 1% error", "≤ 5% error"]


if "selected_features" in df.columns:
    df["selected_features"] = df["selected_features"].apply(lambda x: "original" if "core_number" in x else "new")

for hyperparameter in hyperparameters:
    # Create a temporary dataframe for plotting, dropping rows where the hyperparameter is null
    plot_df = df

    # Filter out some values
    # plot_df = plot_df[~plot_df["pool"].isin(["median"])]
    # # plot_df = plot_df[plot_df["gnn_layers"] == 5]
    # plot_df = plot_df[plot_df["hidden_channels"] == 64]

    # Get the actual categories present in the data
    actual_categories = sorted(plot_df[hyperparameter].unique())

    # Create subplots
    fig_to_show = make_subplots(
        rows=1, cols=2,
        subplot_titles=(f"Accuracy vs. {hyperparameter}", f"Training duration vs {hyperparameter}"),
        horizontal_spacing=0.1
    )
    fig_to_save = go.Figure()

    # Add main metrics to left subplot
    for i, metric in enumerate(metrics_to_plot):
        trace = go.Box(x=plot_df[hyperparameter], y=plot_df[metric],
                       name=metric, boxmean=True,
                       marker_color=px.colors.qualitative.Plotly[i],
                       offsetgroup=str(i))
        fig_to_show.add_trace(trace, row=1, col=1)
        fig_to_save.add_trace(trace)

    # Add duration metric to right subplot
    fig_to_show.add_trace(
        go.Box(x=plot_df[hyperparameter], y=plot_df['duration'],
               name='duration', boxmean=True,
               marker_color=px.colors.qualitative.Plotly[2],
               showlegend=False,
               offsetgroup='duration',
               width=0.6),
        row=1, col=2
    )

    # Update layout for both subplots on the showing figure
    fig_to_show.update_xaxes(
        type='category',
        categoryorder='array',
        categoryarray=actual_categories,
        row=1, col=1
    )
    fig_to_show.update_xaxes(
        type='category',
        categoryorder='array',
        categoryarray=actual_categories,
        row=1, col=2
    )
    fig_to_show.update_yaxes(title_text="% of graphs within error", row=1, col=1)
    fig_to_show.update_yaxes(title_text="Duration (s)", row=1, col=2)

    fig_to_show.update_layout(
        height=400,
        margin=dict(l=20, r=20, t=20, b=20),
        boxmode='group',
        boxgap=0.2,
        boxgroupgap=0.1
    )
    fig_to_show.show()

    # Update layout for the saving figure
    fig_to_save.update_xaxes(
        type='category',
        categoryorder='array',
        categoryarray=actual_categories,
    )
    fig_to_save.update_yaxes(title_text="% of graphs within error")
    fig_to_save.update_layout(
        height=600,
        width=800,
        margin=dict(l=5, r=5, t=5, b=5),
        font=dict(size=20),
        boxmode='group',
        boxgap=0.2,
        boxgroupgap=0.1,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
        ),
        # template="plotly_white"
    )

    if hyperparameter in boxplots_to_save:
        file_path = results_root / "boxplots"
        file_path.mkdir(parents=True, exist_ok=True)
        fig_to_save.write_image(file_path / f"{sweep}_{hyperparameter}.pdf")

# Best model for specified hyperparameter value

In [ ]:
hyperparameter = "selected_features"
metric = "≤ 1% error"

best_runs = df.loc[df.groupby(hyperparameter)[metric].idxmax()]
best_runs.head()

# Statistical tests

In [ ]:
stat_df = df.rename(columns={"≤ 1% error": "Accuracy"})
stat_df = stat_df[hyperparameters + ["Accuracy"]]
stat_df = stat_df[[col for col in stat_df.columns if stat_df[col].nunique() > 1]]

## Paired comparison of effects

In [ ]:
import itertools
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests

def bootstrap_ci(data, n_bootstrap=5000, ci=95, random_state=42):
    rng = np.random.default_rng(random_state)
    boot_means = [rng.choice(data, size=len(data), replace=True).mean() for _ in range(n_bootstrap)]
    lower = np.percentile(boot_means, (100-ci)/2)
    upper = np.percentile(boot_means, 100-(100-ci)/2)
    return lower, upper

all_results = []

for hyperparam in [c for c in stat_df.columns if c != "Accuracy"]:
    values = stat_df[hyperparam].unique()
    for a, b in itertools.combinations(values, 2):
        mask_a = stat_df[hyperparam] == a
        mask_b = stat_df[hyperparam] == b
        merged = stat_df[mask_a].merge(
            stat_df[mask_b],
            on=[c for c in stat_df.columns if c not in [hyperparam, "Accuracy"]],
            suffixes=(f"_{a}", f"_{b}")
        )
        if merged.empty:
            continue
        diff = merged[f"Accuracy_{a}"] - merged[f"Accuracy_{b}"]

        mean_diff = diff.mean()
        median_diff = diff.median()
        std_diff = diff.std()
        ci_low, ci_high = bootstrap_ci(diff.values)
        # effect size: rank-biserial correlation
        effect_size = (np.sum(diff > 0) - np.sum(diff < 0)) / len(diff)

        # Wilcoxon test
        stat, pval = wilcoxon(diff)

        all_results.append({
            "hyperparam": hyperparam,
            "comparison": f"{a} vs {b}",
            "mean_diff": mean_diff,
            "median_diff": median_diff,
            "std_diff": std_diff,
            "ci": [round(ci_low, 2), round(ci_high, 2)],
            "effect_size": effect_size,
            "pval": pval
        })

results_df = pd.DataFrame(all_results)

# Apply Holm correction separately within each hyperparameter
results_df["pval_holm"] = results_df.groupby("hyperparam")["pval"].transform(
    lambda p: multipletests(p, method="holm")[1]
)

print(results_df)


# all_stats_df = pd.concat([all_stats_df, results_df], ignore_index=False)

In [ ]:
def format_dataframe(df, sci_cols):
    df_formatted = df.copy()
    for col in df_formatted.columns:
        if col in sci_cols:
            df_formatted[col] = df_formatted[col].map(lambda x: f'{x:.2e}' if pd.notnull(x) else x)
        elif pd.api.types.is_numeric_dtype(df_formatted[col]):
            df_formatted[col] = df_formatted[col].map(lambda x: f'{x:.2f}' if pd.notnull(x) else x)
    return df_formatted

# all_stats_df = format_dataframe(all_stats_df, sci_cols=['pval', 'pval_holm'])
# all_stats_df.to_csv("../results/hyper_search_stats.csv")

## Importance of hyperparameters

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from itertools import combinations

main_effects = [c for c in stat_df.columns if c != "Accuracy"]
interaction_terms = [f"{a}*{b}" for a, b in combinations(main_effects, 2)]

formula = "Accuracy ~ " + " + ".join(main_effects + interaction_terms)
model = smf.ols(formula, data=stat_df).fit()

anova_table = sm.stats.anova_lm(model, typ=2)
anova_table["eta_sq"] = anova_table["sum_sq"] / anova_table["sum_sq"].sum()
anova_table = anova_table.sort_values(by=["eta_sq"], ascending=False)

print(anova_table)

## Interaction plots

In [ ]:
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

def plot_interaction_grid(df, params, response="Accuracy"):
    pairs = list(itertools.combinations(params, 2))
    n = len(pairs)
    ncols = 3  # how many plots per row
    nrows = -(-n // ncols)  # ceil division

    fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 4*nrows), squeeze=False)

    for ax, (p1, p2) in zip(axes.flatten(), pairs):
        sns.pointplot(
            data=df, x=p1, y=response, hue=p2,
            dodge=True, markers="o", capsize=0.1, errorbar="pi", ax=ax  # pi = percentile interval
        )
        ax.set_title(f"{p1} × {p2}")
        ax.legend(title=p2, fontsize="small", title_fontsize="small")

    # Remove empty subplots if any
    for ax in axes.flatten()[len(pairs):]:
        ax.remove()

    plt.tight_layout()
    plt.show()

# Example usage:
params = [c for c in stat_df.columns if c != "Accuracy"]
plot_interaction_grid(stat_df, params)

In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.colors

def save_interaction_plot(df, factor1, factor2, response, results_root, sweep):
    fig = go.Figure()

    if not factor1 or not factor2 or factor1 not in df.columns or factor2 not in df.columns:
        return

    # Get unique values for factors
    x_categories = sorted(df[factor1].unique())
    hue_values = sorted(df[factor2].unique())

    # --- COLOR LOGIC ---
    is_hue_numeric = pd.api.types.is_numeric_dtype(df[factor2])
    if is_hue_numeric:
        hue_min, hue_max = df[factor2].min(), df[factor2].max()

        color_map = {}
        for value in hue_values:
            if hue_min == hue_max:
                norm_val = 0.5
            else:
                norm_val = (value - hue_min) / (hue_max - hue_min)
            color_map[value] = plotly.colors.sample_colorscale('burg', norm_val)[0]
    else:
        qualitative_colors = px.colors.qualitative.Plotly
        color_map = {hue: qualitative_colors[i % len(qualitative_colors)] for i, hue in enumerate(hue_values)}
    # --- END COLOR LOGIC ---

    # Define an offset for dodging
    num_hues = len(hue_values)
    offset_width = 0.05
    offsets = np.linspace(0, offset_width * num_hues, num_hues) if num_hues > 1 else [0]

    # Create traces for each hue value
    for i, hue in enumerate(hue_values):
        means = []
        errors = []

        x_numeric = np.arange(len(x_categories))
        x_offset = x_numeric + offsets[i]

        for cat in x_categories:
            subset = df[(df[factor1] == cat) & (df[factor2] == hue)]
            if not subset.empty:
                mean = subset[response].mean()
                means.append(mean)
                lower, upper = np.percentile(subset[response], [2.5, 97.5])
                errors.append(dict(upper=upper - mean, lower=mean - lower))
            else:
                means.append(None)
                errors.append(dict(upper=0, lower=0))

        fig.add_trace(go.Scatter(
            x=x_offset,
            y=means,
            mode='lines+markers',
            name=str(hue),
            line=dict(width=3, color=color_map[hue]),
            marker=dict(size=10, color=color_map[hue]),
            error_y=dict(
                type='data',
                symmetric=False,
                array=[e['upper'] for e in errors],
                arrayminus=[e['lower'] for e in errors],
                visible=True,
                width=7,
                thickness=3,
            )
        ))

    fig.update_layout(
        xaxis_title=factor1,
        yaxis_title=response,
        height=600,
        width=800,
        margin=dict(l=5, r=5, t=40, b=70),
        font=dict(size=20),
        xaxis=dict(
            tickmode='array',
            tickvals=np.arange(len(x_categories)),
            ticktext=x_categories
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            title=factor2
        ),
    )

    image_path = results_root / "interaction_plots"
    image_path.mkdir(parents=True, exist_ok=True)
    image_path /= f"{sweep}_{factor1}_{factor2}.pdf"
    fig.write_image(image_path)


# Replace the old function call
for factor1, factor2 in interaction_plots_to_save:
    save_interaction_plot(stat_df, factor1, factor2, "Accuracy", results_root, sweep)
